In [ ]:
Data  :  https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [647]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

In [648]:
train = pd.read_csv('./data/spam.csv',  encoding='latin-1')

In [649]:
train.head(100)
train = train.iloc[:,:2]

In [650]:
Ytrain = train.iloc[:,:1]
Xtrain = train.iloc[:,1:]

In [651]:
Xtrain['length'] = Xtrain['v2'].apply(len)

In [652]:
def cleaner(message):
    #message = message.translate(str.maketrans('', '', string.punctuation))
    word = [(word ) for word in message.split() if word not in stopwords.words('english')]
    return " ".join(word)

def Yvalue_classification(mess):
    if mess == 'spam':
        return 1
    else :
        return 0

In [653]:
Xtrain['v2'] = Xtrain['v2'].apply(cleaner)

In [654]:
Ytrain['v1'] = Ytrain['v1'].apply(Yvalue_classification)

D:\Tools\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [655]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('sigmoid')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [656]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xtrain, Ytrain, stratify = Ytrain['v1'],test_size = 0.2)

In [657]:
max_words = 1000
max_len = 150

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xtrain['v2'])
sequences = tok.texts_to_sequences(Xtrain['v2'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [658]:
model.fit(sequences_matrix,Ytrain,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3565 samples, validate on 892 samples
Epoch 1/10
3565/3565 [==============================] - 13s 4ms/step - loss: 0.3352 - acc: 0.8760 - val_loss: 0.2296 - val_acc: 0.9753
Epoch 2/10
3565/3565 [==============================] - 10s 3ms/step - loss: 0.1147 - acc: 0.9658 - val_loss: 0.1186 - val_acc: 0.9809
Epoch 3/10
3565/3565 [==============================] - 10s 3ms/step - loss: 0.0615 - acc: 0.9849 - val_loss: 0.0632 - val_acc: 0.9865
Epoch 4/10
3565/3565 [==============================] - 9s 3ms/step - loss: 0.0358 - acc: 0.9882 - val_loss: 0.0592 - val_acc: 0.9854
Epoch 5/10
3565/3565 [==============================] - 9s 3ms/step - loss: 0.0313 - acc: 0.9902 - val_loss: 0.0648 - val_acc: 0.9854


In [659]:
test_sequences = tok.texts_to_sequences(Xtest['v2'])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Ytest)
print('Loss: {}\n  Accuracy: {}'.format(accr[0],accr[1]))

1115/1115 [==============================] - 0s 353us/step
Test set
  Loss: 0.046
  Accuracy: 0.988
